In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#import dataset from weather_data.csv file
dataset = pd.read_csv("content\weather_data.csv", sep = ";")
dataset=dataset.reset_index(drop=True)
training_set = dataset.iloc[:6000,2:3].values


In [2]:
#Feature Scaling 
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)

In [3]:
x_train = []
y_train = []
n_future = 8 # next 24 hours temperature forecast
n_past = 56 # Past 7 days
for i in range(0,len(training_set_scaled)-n_past-n_future+1):
    x_train.append(training_set_scaled[i : i + n_past , 0])     
    y_train.append(training_set_scaled[i + n_past : i + n_past + n_future , 0 ])
x_train , y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0] , x_train.shape[1], 1) )

In [4]:
from keras.models import Sequential
from keras.layers import LSTM,Dense ,Dropout
from tensorflow.keras.layers import Bidirectional
# Fitting RNN to training set using Keras Callbacks. Read Keras callbacks docs for more info.

In [5]:
regressor = Sequential()
regressor.add(Bidirectional(LSTM(units=30, return_sequences=True, input_shape = (x_train.shape[1],1) ) ))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30 , return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30 , return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = n_future,activation='linear'))
regressor.compile(optimizer='adam', loss='mean_squared_error',metrics=['acc'])
regressor.fit(x_train, y_train, epochs=30,batch_size=32 )

Epoch 1/30
186/186 [==============================] - 16s 48ms/step - loss: 0.0412 - acc: 0.1315
Epoch 2/30
186/186 [==============================] - 8s 44ms/step - loss: 0.0285 - acc: 0.1300
Epoch 3/30
186/186 [==============================] - 8s 44ms/step - loss: 0.0263 - acc: 0.1395
Epoch 4/30
186/186 [==============================] - 8s 42ms/step - loss: 0.0213 - acc: 0.2392
Epoch 5/30
186/186 [==============================] - 9s 47ms/step - loss: 0.0180 - acc: 0.2904
Epoch 6/30
186/186 [==============================] - 9s 48ms/step - loss: 0.0143 - acc: 0.3935
Epoch 7/30
186/186 [==============================] - 8s 44ms/step - loss: 0.0104 - acc: 0.5191
Epoch 8/30
186/186 [==============================] - 8s 45ms/step - loss: 0.0085 - acc: 0.5936
Epoch 9/30
186/186 [==============================] - 8s 44ms/step - loss: 0.0076 - acc: 0.6476
Epoch 10/30
186/186 [==============================] - 8s 45ms/step - loss: 0.0068 - acc: 0.6913
Epoch 11/30
186/186 [=================

In [6]:
# read test dataset
testdataset = pd.read_csv('content\weather_data.csv', sep =";")
testdataset = testdataset.drop("time", axis = 1)
testdataset = testdataset.drop("direction", axis = 1)
testdataset = testdataset.drop("weather", axis = 1)
#get only the temperature column
testdataset = testdataset.iloc[6000:8000,2:3].values
real_temperature = pd.read_csv('content\weather_data.csv', sep = ";")
real_temperature = real_temperature.iloc[8000:,2:3].values
testing = sc.transform(testdataset)
testing = np.array(testing)
testing = np.reshape(testing,(testing.shape[1],testing.shape[0],1))


In [7]:
from sklearn.metrics import accuracy_score
import sklearn
predicted_temperature = regressor.predict(testing)
predicted_temperature = sc.inverse_transform(predicted_temperature)
predicted_temperature = np.reshape(predicted_temperature,(predicted_temperature.shape[1],predicted_temperature.shape[0],1))

1/1 [==============================] - 2s 2s/step
